In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.ensemble import VotingClassifier

In [ ]:
#Importar datos
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Lee el archivo Excel en un DataFrame de pandas
datos = pd.read_excel(file_name)

Saving Datos_iefic.xlsx to Datos_iefic.xlsx


In [ ]:
logit = LogisticRegression(solver='lbfgs')
xgb = XGBClassifier(objective='binary:logistic')
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='logistic')

In [ ]:
#Definición de la variable dependiente e independeientes
x13 = datos.drop("impago", axis=1)
y13 = datos["impago"]

In [ ]:
#Rebalanceo
#95% -> 0 / 5% -> 1 | 65%-35%
# Se hace un oversampling para balancear la muestra
#Oversampling a los 1 (Fail) para alcanzar un 35%
oversampler = RandomOverSampler(sampling_strategy=0.35, random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(x13, y13)

#Undersampling a los 0 (Current) para alcanzar un 65%
undersampler = RandomUnderSampler(sampling_strategy=0.65, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_resampled, y_resampled)

------------------- Red neuronal multicapa --------------------

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Crear y entrenar una red neuronal MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='logistic', max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_mlp1 = mlp.predict(X_test)

# Imprimir un informe de clasificación
report = classification_report(y_test, y_pred_mlp1)
print(report)

              precision    recall  f1-score   support

           0       0.66      0.97      0.78      6401
           1       0.82      0.22      0.35      4191

    accuracy                           0.67     10592
   macro avg       0.74      0.60      0.57     10592
weighted avg       0.72      0.67      0.61     10592



------------------------------XGB-----------------------------

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Búsqueda de hiperparámetros
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 7),
    'learning_rate': uniform(0.01, 0.2),
    'subsample': uniform(0.6, 0.3),
    'colsample_bytree': uniform(0.6, 0.3)
}

# Crear un objeto XGBoost para clasificación binaria
xgb_model = XGBClassifier(objective='binary:logistic')

# Realizar la búsqueda aleatoria de hiperparámetros con menos iteraciones
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=20, cv=5, scoring='precision', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = random_search.best_params_

# Utilizar el modelo xgb_model con los mejores hiperparámetros
xgb_model.set_params(**best_params)

# Entrenar el modelo XGBoost con los mejores hiperparámetros
xgb_model.fit(X_train, y_train)

# Realizar predicciones en tus datos de prueba
y_pred_xgb = xgb_model.predict(X_test)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [ ]:
print(best_params)

{'colsample_bytree': 0.8313811040057837, 'learning_rate': 0.024808930346818074, 'max_depth': 5, 'n_estimators': 140, 'subsample': 0.8744879026631343}


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import randint, uniform
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Nuevos valores de hiperparámetros
new_params = {
    'n_estimators': 80,
    'max_depth': 2,
    'learning_rate': 0.01,
    'subsample': 0.6,
    'colsample_bytree': 0.6
}

# Actualizar el modelo XGBoost con los nuevos hiperparámetros
xgb_model.set_params(**new_params)

# Entrenar el modelo XGBoost con los nuevos hiperparámetros
xgb_model.fit(X_train, y_train)

# Realizar predicciones en tus datos de prueba
y_pred1 = xgb_model.predict(X_test)

# Calcular métricas
accuracy = accuracy_score(y_test, y_pred1)
precision = precision_score(y_test, y_pred1)
recall = recall_score(y_test, y_pred1)
f1 = f1_score(y_test, y_pred1)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')


Accuracy: 0.9336291540785498
Precision: 1.0
Recall: 0.8322596039131472
F1-Score: 0.9084516213048575


-------------------- Random Forest -----------------------

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
#Entrenar modelo
rf.fit(X_train, y_train)
#Calibrar hiperparametros
# Definir la cuadrícula de hiperparámetros a buscar
param_grid = {
    'n_estimators': [100, 120 ,150, 200],
    'max_depth': [5, 10, 25, 50],
}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')

# Realizar la búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_

print(best_params)

y_pred = rf.predict(X_test)

# Calcular las métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Imprimir los resultados
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')

{'max_depth': 25, 'n_estimators': 120}
Accuracy: 0.9948074018126888
Precision: 0.9930853600381497
Recall: 0.9937962300167025
F1-Score: 0.9934406678592725


-------------- Logit -----------------

In [ ]:
# Definir hiperparámetros a calibrar
logit_params = {'C': [0.1, 1, 5,10, 100]}

# Crear un modelo de regresión logística
logit = LogisticRegression()

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Crear una instancia de StratifiedKFold para la validación cruzada
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Realizar la búsqueda de hiperparámetros
logit_grid = GridSearchCV(logit, logit_params, cv=skf, scoring='accuracy')
logit_grid.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros para Regresión Logística:", logit_grid.best_params_)

# Obtener las probabilidades predichas en el conjunto de prueba
y_pred_prob = logit_grid.predict_proba(X_test)[:, 1]

# Definir una lista de umbrales
thresholds = [0.2, 0.3, 0.4, 0.5, 0.6]

# Iterar sobre los umbrales y evaluar el rendimiento
for threshold in thresholds:
    y_pred = (y_pred_prob > threshold).astype(int)

    # Calcular métricas de rendimiento
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    print(f"Umbral: {threshold}, Precisión: {precision}, Recall: {recall}, Exactitud: {accuracy}")

Mejores hiperparámetros para Regresión Logística: {'C': 0.1}
Umbral: 0.2, Precisión: 0.39583135954523924, Recall: 0.9968981150083512, Exactitud: 0.396714501510574
Umbral: 0.3, Precisión: 0.3963903743315508, Recall: 0.9904557384872346, Exactitud: 0.399452416918429
Umbral: 0.4, Precisión: 0.3948306595365419, Recall: 0.9513242662848962, Exactitud: 0.4037953172205438
Umbral: 0.5, Precisión: 0.7322654462242563, Recall: 0.3054163684084944, Exactitud: 0.6809856495468278
Umbral: 0.6, Precisión: 0.6398601398601399, Recall: 0.043664996420901936, Exactitud: 0.6118768882175226


----------- Modelos de ensamble -----------

Voting

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
import numpy as np
from scipy import stats

# Dividir tus datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Obtener las predicciones de cada modelo
pred_logit = y_pred_prob
pred_mlp = y_pred_mlp1
pred_xgb = y_pred_xgb

# Crear un ensemble de votación
voting_model = VotingClassifier(estimators=[
    ('logit', pred_logit),
    ('mlp', pred_mlp),
    ('xgb', pred_xgb)
], voting='hard')

# Crea una matriz con las predicciones de tus modelos
predictions_matrix = np.column_stack((pred_logit, pred_mlp, pred_xgb))

y_pred_voting = stats.mode(predictions_matrix, axis=1).mode.ravel()


In [ ]:
y_true = y_test  # Reemplaza 'y_test' con tus etiquetas reales

# Calcular métricas de rendimiento para 'y_pred_voting'
accuracy = accuracy_score(y_true, y_pred_voting)
precision = precision_score(y_true, y_pred_voting)
recall = recall_score(y_true, y_pred_voting)
f1 = f1_score(y_true, y_pred_voting)

# Imprimir las métricas
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')

Accuracy: 0.6907099697885196
Precision: 0.9956663055254604
Recall: 0.2192794082557862
F1-Score: 0.35940555338287056


Stacking

In [ ]:
#Stacking
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy import stats

# Dividir tus datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# 1. Entrena tus modelos base
logit = LogisticRegression(solver='lbfgs')
logit.fit(X_train, y_train)

mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='logistic', max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

# Nuevos valores de hiperparámetros para XGB (como en tu código original)
new_params = {
    'n_estimators': 80,
    'max_depth': 2,
    'learning_rate': 0.01,
    'subsample': 0.6,
    'colsample_bytree': 0.6
}

xgb_model = XGBClassifier(objective='binary:logistic', **new_params)
xgb_model.fit(X_train, y_train)

# 2. Obtener las predicciones de cada modelo
pred_logit = logit.predict(X_test)
pred_mlp = mlp.predict(X_test)
pred_xgb = xgb_model.predict(X_test)

# Crear un ensemble con un modelo de votación
estimators = [
    ('logit', logit),
    ('mlp', mlp),
    ('xgb', xgb_model)
]

# Crear un modelo de apilamiento (StackingClassifier) con un modelo meta (en este caso, Logistic Regression)
stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

# Entrenar el modelo de apilamiento en el conjunto de entrenamiento
stacking_model.fit(X_train, y_train)

# Realizar predicciones con el modelo de apilamiento en el conjunto de prueba
y_pred_stacking = stacking_model.predict(X_test)

# Calcular métricas de rendimiento para 'y_pred_stacking'
accuracy = accuracy_score(y_test, y_pred_stacking)
precision = precision_score(y_test, y_pred_stacking)
recall = recall_score(y_test, y_pred_stacking)
f1 = f1_score(y_test, y_pred_stacking)

# Imprimir las métricas
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')


Accuracy: 0.947129909365559
Precision: 0.9913396481732071
Recall: 0.8740157480314961
F1-Score: 0.9289880801420238


Promedios

In [ ]:
from sklearn.metrics import accuracy_score

# Obtener las predicciones de cada modelo
y_pred_logit = logit_grid.predict(X_test)
y_pred_mlp = mlp.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)

# Calcular las precisiones de cada modelo
accuracy_logit = accuracy_score(y_test, y_pred_logit)
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)

# Calcular el promedio de las precisiones
average_accuracy = (accuracy_logit + accuracy_mlp + accuracy_xgb) / 4

print(f'Accuracy promedio de los modelos: {average_accuracy}')

from sklearn.metrics import confusion_matrix

# Obtener las predicciones de cada modelo
y_pred_logit = logit_grid.predict(X_test)
y_pred_mlp = mlp.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)

# Crear un arreglo que contenga todas las predicciones
all_predictions = [y_pred_logit, y_pred_mlp, y_pred_xgb]

# Crear una matriz de confusión combinando las predicciones de todos los modelos
combined_predictions = np.mean(all_predictions, axis=0)  # Promedio de las predicciones

# Redondear las predicciones combinadas a 0 o 1
combined_predictions = np.round(combined_predictions)

# Calcular la matriz de confusión para las predicciones combinadas
confusion = confusion_matrix(y_test, combined_predictions)

# Imprimir la matriz de confusión
print("Matriz de Confusión Combinada:")
print(confusion)
tn = confusion[0, 0] #Verdaderos negativos
fn = confusion[1, 0] #falsos negativos
fp = confusion[0, 1] #falsos positivos
tp = confusion[1, 1] #Verdaderos positivos

sensitividad = tp / (tp + fp) #sensitividad
print(sensitividad)
especificidad = tn / (tn + fn) #especificidad
print(especificidad)
fal_neg = fn / (fn + tn)
print(fal_neg)
fal_pos = fp / (fp + tp)
print(fal_pos)

Accuracy promedio de los modelos: 0.5720118957703928
Matriz de Confusión Combinada:
[[6231  170]
 [2861 1330]]
0.8866666666666667
0.6853277606687197
0.31467223933128025
0.11333333333333333


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Entrenar y evaluar el modelo XGBoost
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# Calcular métricas para el modelo XGBoost
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)

# Imprimir métricas para el modelo XGBoost
print("XGBoost Metrics:")
print(f'Accuracy: {accuracy_xgb}')
print(f'Precision: {precision_xgb}')
print(f'Recall: {recall_xgb}')
print(f'F1-Score: {f1_xgb}')

# Entrenar y evaluar el modelo de Regresión Logística
logit = LogisticRegression()
logit.fit(X_train, y_train)
y_pred_logit = logit.predict(X_test)

# Calcular métricas para el modelo de Regresión Logística
accuracy_logit = accuracy_score(y_test, y_pred_logit)
precision_logit = precision_score(y_test, y_pred_logit)
recall_logit = recall_score(y_test, y_pred_logit)
f1_logit = f1_score(y_test, y_pred_logit)

# Imprimir métricas para el modelo de Regresión Logística
print("Logistic Regression Metrics:")
print(f'Accuracy: {accuracy_logit}')
print(f'Precision: {precision_logit}')
print(f'Recall: {recall_logit}')
print(f'F1-Score: {f1_logit}')

# Entrenar y evaluar la Red Neuronal MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='logistic', max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)

# Calcular métricas para la Red Neuronal MLP
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
precision_mlp = precision_score(y_test, y_pred_mlp)
recall_mlp = recall_score(y_test, y_pred_mlp)
f1_mlp = f1_score(y_test, y_pred_mlp)

# Imprimir métricas para la Red Neuronal MLP
print("MLP Metrics:")
print(f'Accuracy: {accuracy_mlp}')
print(f'Precision: {precision_mlp}')
print(f'Recall: {recall_mlp}')
print(f'F1-Score: {f1_mlp}')

# Calcular el promedio de métricas
avg_accuracy = (accuracy_xgb + accuracy_logit + accuracy_mlp) / 3
avg_precision = (precision_xgb + precision_logit + precision_mlp) / 3
avg_recall = (recall_xgb + recall_logit + recall_mlp) / 3
avg_f1 = (f1_xgb + f1_logit + f1_mlp) / 3

# Imprimir el promedio de métricas
print("Promedio de Métricas:")
print(f'Accuracy: {avg_accuracy}')
print(f'Precision: {avg_precision}')
print(f'Recall: {avg_recall}')
print(f'F1-Score: {avg_f1}')


XGBoost Metrics:
Accuracy: 0.9913141993957704
Precision: 0.9944511459589868
Recall: 0.9835361488904796
F1-Score: 0.9889635316698656
Logistic Regression Metrics:
Accuracy: 0.6809856495468278
Precision: 0.7322654462242563
Recall: 0.3054163684084944
F1-Score: 0.43104899814783637
MLP Metrics:
Accuracy: 0.6734327794561934
Precision: 0.823321554770318
Recall: 0.22238129324743497
F1-Score: 0.35017847078715003
Promedio de Métricas:
Accuracy: 0.7819108761329305
Precision: 0.8500127156511871
Recall: 0.503777936848803
F1-Score: 0.590063666868284
